In [8]:
!pip install dspy-ai pandas

In [9]:
!pip install python-dotenv openai

In [10]:

# Set environment variables directly in Colab
import os
os.environ['OPENROUTER_API_KEY'] = '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'
os.environ['MODEL'] = 'z-ai/glm-4.5-air:free'
os.environ['TEMPERATURE'] = '0.7'

In [ ]:
#Their are 2 methods. The below cell uses "Chain of Thought" and other cell below it uses "Predict". The first one is more sophisticted. play with temperature change.

In [19]:
import os
import logging
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv('/content/.env')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import required libraries
import dspy
from dspy import Example, Predict, ChainOfThought

# API configuration
API_KEY = os.getenv("OPENROUTER_API_KEY") or ""
MODEL = "z-ai/glm-4.5-air:free"
TEMPERATURE = 0.7

# Check if API key is available
if not API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")

# Set up DSPy to use GLM via OpenRouter
lm = dspy.LM(
    model="openai/" + MODEL,
    api_key=API_KEY,
    temperature=TEMPERATURE,
    base_url="https://openrouter.ai/api/v1"
)
dspy.settings.configure(lm=lm)
logger.info(f"Successfully configured DSPy with {MODEL}")

# Import validation functions
def validate_taste(taste: str) -> str:
    """Validate the artistic taste category."""
    valid_tastes = ['photorealistic', 'oil painting', 'anime', 'cyberpunk', 'watercolor', '3d render']
    if taste.lower() not in valid_tastes:
        raise ValueError(f"Invalid taste. Choose from: {', '.join(valid_tastes)}")
    return taste.lower()

def validate_input(user_input: str) -> str:
    """Validate the user input description."""
    if not user_input or len(user_input.strip()) < 3:
        raise ValueError("Please provide a description (at least 3 characters)")
    return user_input.strip()

# Define DSPy signatures
class InitialPromptOptimization(dspy.Signature):
    """Optimize user input into a detailed image prompt for the specified artistic style."""
    taste = dspy.InputField(desc="Artistic style (e.g., photorealistic, anime)")
    user_input = dspy.InputField(desc="Simple user description")
    optimized_prompt = dspy.OutputField(desc="Detailed optimized prompt")
    reasoning = dspy.OutputField(desc="Step-by-step reasoning for the optimization")

class PromptEnhancement(dspy.Signature):
    """Enhance an optimized prompt with technical details and artistic refinements."""
    base_prompt = dspy.InputField(desc="Already optimized prompt")
    taste = dspy.InputField(desc="Artistic style")
    enhanced_prompt = dspy.OutputField(desc="Enhanced prompt with technical details and artistic refinements")
    reasoning = dspy.OutputField(desc="Step-by-step reasoning for the enhancement")

# Define DSPy modules
class InitialPromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        # Changed from Predict to ChainOfThought
        self.optimize = ChainOfThought(InitialPromptOptimization)

    def forward(self, taste: str, user_input: str):
        return self.optimize(taste=taste, user_input=user_input)

class PromptEnhancer(dspy.Module):
    def __init__(self):
        super().__init__()
        # Changed from Predict to ChainOfThought
        self.enhance = ChainOfThought(PromptEnhancement)

    def forward(self, base_prompt: str, taste: str):
        return self.enhance(base_prompt=base_prompt, taste=taste)

class FullPromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.initial_optimizer = InitialPromptOptimizer()
        self.enhancer = PromptEnhancer()

    def forward(self, taste: str, user_input: str):
        # Step 1: Initial prompt optimization
        optimized_result = self.initial_optimizer(taste=taste, user_input=user_input)

        # Step 2: Enhance the optimized prompt
        enhanced_result = self.enhancer(
            base_prompt=optimized_result.optimized_prompt,
            taste=taste
        )

        # Return the results as attributes of a simple object
        # Note: ChainOfThought returns 'reasoning' not 'optimized_reasoning' and 'enhanced_reasoning'
        class OptimizationResult:
            def __init__(self, optimized_prompt, reasoning, enhanced_prompt, enhanced_reasoning):
                self.optimized_prompt = optimized_prompt
                self.reasoning = reasoning
                self.enhanced_prompt = enhanced_prompt
                self.enhanced_reasoning = enhanced_reasoning

        return OptimizationResult(
            optimized_prompt=optimized_result.optimized_prompt,
            reasoning=optimized_result.reasoning,
            enhanced_prompt=enhanced_result.enhanced_prompt,
            enhanced_reasoning=enhanced_result.reasoning
        )

# Prepare training data
def create_training_data() -> List[Dict[str, str]]:
    """Create training examples for DSPy."""
    return [
        {
            'taste': 'photorealistic',
            'user_input': 'a cat sitting on a windowsill',
            'optimized_prompt': 'Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture, sharp focus, natural lighting, 85mm lens, shallow depth of field',
            'optimized_reasoning': 'The original description is simple. I enhanced it by adding details about the cat\'s color, the lighting conditions, camera equipment, and photography techniques.',
            'enhanced_prompt': 'Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture, sharp focus, natural lighting, 85mm lens, shallow depth of field, 4K resolution, f/1.8 aperture, golden hour glow, cinematic composition',
            'enhanced_reasoning': 'I added technical photography details like resolution, aperture, and lighting to make the prompt more specific for photorealistic rendering.'
        },
        {
            'taste': 'oil painting',
            'user_input': 'a mountain landscape',
            'optimized_prompt': 'Oil painting of majestic mountain landscape at sunset, impressionist style, visible brushstrokes, warm golden hour light, textured canvas, rich color palette',
            'optimized_reasoning': 'I enhanced the basic mountain landscape by specifying it as an oil painting in impressionist style, with details about brushwork and lighting.',
            'enhanced_prompt': 'Oil painting of majestic mountain landscape at sunset, impressionist style, visible brushstrokes, warm golden hour light, textured canvas, rich color palette, canvas texture visible, Rembrandt lighting, thick impasto technique, 24"x36" aspect ratio',
            'enhanced_reasoning': 'I added oil painting specific techniques like impasto, canvas texture, and referenced Rembrandt lighting to enhance the artistic quality.'
        },
        {
            'taste': 'anime',
            'user_input': 'a futuristic city',
            'optimized_prompt': 'Anime style illustration of a futuristic cyberpunk cityscape at night, neon lights, flying vehicles, detailed architecture, vibrant colors, high contrast',
            'optimized_reasoning': 'I took the basic futuristic city concept and transformed it into an anime-style cyberpunk city with specific visual elements.',
            'enhanced_prompt': 'Anime style illustration of a futuristic cyberpunk cityscape at night, neon lights, flying vehicles, detailed architecture, vibrant colors, high contrast, Studio Ghibli influences, detailed backgrounds, cel-shaded rendering, 1080p resolution',
            'enhanced_reasoning': 'I added anime-specific rendering techniques like cel-shading, referenced Studio Ghibli style, and included resolution details.'
        },
        {
            'taste': 'cyberpunk',
            'user_input': 'a street market',
            'optimized_prompt': 'Cyberpunk street market at night, neon signs, diverse crowd, stalls selling futuristic tech, rain-slicked streets, Blade Runner aesthetic',
            'optimized_reasoning': 'I enhanced the basic street market with cyberpunk elements like neon lighting, futuristic technology, and rain-slicked streets.',
            'enhanced_prompt': 'Cyberpunk street market at night, neon signs, diverse crowd, stalls selling futuristic tech, rain-slicked streets, Blade Runner aesthetic, cinematic lighting, volumetric fog, reflections on wet pavement, ultra-detailed, 8K resolution',
            'enhanced_reasoning': 'I added cinematic elements like volumetric fog and reflections, and included technical details like 8K resolution for higher quality rendering.'
        }
    ]

# Simple data container
class TrainingData:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return Example(
            taste=item['taste'],
            user_input=item['user_input'],
            optimized_prompt=item['optimized_prompt'],
            optimized_reasoning=item['optimized_reasoning'],
            enhanced_prompt=item['enhanced_prompt'],
            enhanced_reasoning=item['enhanced_reasoning']
        ).with_inputs('taste', 'user_input')

# Quality metric for DSPy evaluation
def prompt_quality_metric(example, pred, trace=None) -> float:
    """Calculate quality score for the final enhanced prompt."""
    prompt = pred.enhanced_prompt

    # Quality indicators
    has_details = 'detailed' in prompt.lower()
    has_style = any(style in prompt.lower() for style in ['style', 'technique', 'aesthetic'])
    has_technical = any(tech in prompt.lower() for tech in ['lighting', 'focus', 'texture', 'composition', 'resolution', 'aperture'])
    has_mood = any(mood in prompt.lower() for mood in ['mood', 'atmosphere', 'ambiance', 'feeling'])

    # Check if it includes relevant technical details for the style
    has_photography_terms = True
    has_painting_terms = True

    if example.taste == 'photorealistic':
        has_photography_terms = any(term in prompt.lower() for term in ['aperture', 'f/', 'shutter', 'depth', 'bokeh'])
    elif example.taste == 'oil painting':
        has_painting_terms = any(term in prompt.lower() for term in ['brushstroke', 'impasto', 'canvas', 'palette', 'texture'])

    # Calculate score
    indicators = [has_details, has_style, has_technical, has_mood, has_photography_terms, has_painting_terms]
    return sum(indicators) / len(indicators)

# Main DSPy-powered prompt generator
class DSPyPromptGenerator:
    def __init__(self):
        # Create training data
        self.trainset = TrainingData(create_training_data())

        # Initialize the full optimizer
        self.optimizer = FullPromptOptimizer()

        # Try to use BootstrapFewShot if available
        try:
            from dspy.teleprompter import BootstrapFewShot
            teleprompter = BootstrapFewShot(metric=prompt_quality_metric)
            self.compiled_optimizer = teleprompter.compile(self.optimizer, trainset=self.trainset)
            logger.info("Using compiled optimizer with BootstrapFewShot")
        except ImportError:
            try:
                from dspy.teleprompters import BootstrapFewShot
                teleprompter = BootstrapFewShot(metric=prompt_quality_metric)
                self.compiled_optimizer = teleprompter.compile(self.optimizer, trainset=self.trainset)
                logger.info("Using compiled optimizer with BootstrapFewShot")
            except ImportError:
                logger.warning("Could not import BootstrapFewShot from teleprompter module")
                logger.info("Using non-compiled optimizer instead")
                self.compiled_optimizer = self.optimizer

        self.results = []

    def generate(self, taste: str, user_input: str) -> str:
      """Generate an enhanced image prompt using DSPy-optimized modules."""
      try:
          # Validate inputs
          taste = validate_taste(taste)
          user_input = validate_input(user_input)
          logger.info(f"Validated input: {taste} - {user_input}")

          # Use the compiled DSPy optimizer or fallback
          logger.info("Running DSPy-optimized prompt generation...")
          result = self.compiled_optimizer(taste=taste, user_input=user_input)

          optimized_prompt = result.optimized_prompt
          optimized_reasoning = result.reasoning  # This exists in ChainOfThought
          final_prompt = result.enhanced_prompt
          final_reasoning = result.enhanced_reasoning  # This exists in ChainOfThought

          # Calculate quality score
          # Create a simple object with the enhanced prompt for the metric function
          class SimplePrediction:
              def __init__(self, enhanced_prompt):
                  self.enhanced_prompt = enhanced_prompt

          quality_score = prompt_quality_metric(
              Example(taste=taste, user_input=user_input),
              SimplePrediction(final_prompt)
          )

          # Store result
          self.results.append({
              'taste': taste,
              'user_input': user_input,
              'optimized_prompt': optimized_prompt,
              'optimized_reasoning': optimized_reasoning,
              'final_prompt': final_prompt,
              'final_reasoning': final_reasoning,
              'quality_score': quality_score
          })

          logger.info(f"Generated prompt with quality score: {quality_score:.2f}")
          logger.info(f"Optimization reasoning: {optimized_reasoning}")
          logger.info(f"Enhancement reasoning: {final_reasoning}")
          return final_prompt

      except Exception as e:
          logger.error(f"Error in prompt generation: {e}")
          fallback_prompt = f"{taste} style image of {user_input}, detailed, high quality"
          logger.info(f"Using fallback prompt: {fallback_prompt}")

          # Also store fallback result
          self.results.append({
              'taste': taste,
              'user_input': user_input,
              'optimized_prompt': fallback_prompt,
              'optimized_reasoning': 'Fallback due to error',
              'final_prompt': fallback_prompt,
              'final_reasoning': 'Fallback due to error',
              'quality_score': 0.0
          })

          return fallback_prompt

    def get_results(self) -> List[Dict[str, Any]]:
        """Get all generated results."""
        return self.results

def main():
    """Main function using actual DSPy."""
    print("=== DSPy-Optimized Image Prompt Generation System ===\n")
    print("Using DSPy with:")
    print("1. Defined signatures for initial optimization and enhancement")
    print("2. Multi-stage pipeline (Initial Optimizer → Enhancer)")
    print("3. Chain-of-Thought reasoning for both optimization steps")

    try:
        from dspy.teleprompter import BootstrapFewShot
        print("4. BootstrapFewShot for automatic prompt optimization")
    except ImportError:
        try:
            from dspy.teleprompters import BootstrapFewShot
            print("4. BootstrapFewShot for automatic prompt optimization")
        except ImportError:
            print("4. Basic prompt optimization (teleprompter module not available)")

    print("5. Quality-based metrics for evaluation")
    print(f"6. GLM-4.5-Air as the backend language model\n")

    # Test examples
    examples = [
        {
            "taste": "oil painting",
            "user_input": "a man climbing mountain"
        }
    ]

    # Initialize the DSPy prompt generator
    generator = DSPyPromptGenerator()

    for i, example in enumerate(examples, 1):
        print(f"Example {i}:")
        print(f"Taste: {example['taste']}")
        print(f"User Input: {example['user_input']}")

        try:
            # Generate using DSPy
            result = generator.generate(
                taste=example['taste'],
                user_input=example['user_input']
            )

            print(f"\nDSPy-Optimized Prompt:")
            print(f"{generator.results[-1]['optimized_prompt']}")

            print(f"\nOptimization Reasoning:")
            print(f"{generator.results[-1]['optimized_reasoning']}")

            print(f"\nDSPy-Enhanced Prompt (Final Output):")
            print(f"{result}")

            print(f"\nEnhancement Reasoning:")
            print(f"{generator.results[-1]['final_reasoning']}")

            print(f"\nQuality Score: {generator.results[-1]['quality_score']:.2f}")
            print("✅ SUCCESS: DSPy-optimized prompt generation completed!\n")

        except Exception as e:
            print(f"❌ FAILED: {e}\n")

        print("="*70)

if __name__ == "__main__":
    main()

=== DSPy-Optimized Image Prompt Generation System ===

Using DSPy with:
1. Defined signatures for initial optimization and enhancement
2. Multi-stage pipeline (Initial Optimizer → Enhancer)
3. Chain-of-Thought reasoning for both optimization steps
4. Basic prompt optimization (teleprompter module not available)
5. Quality-based metrics for evaluation
6. GLM-4.5-Air as the backend language model

Example 1:
Taste: oil painting
User Input: a man climbing mountain

DSPy-Optimized Prompt:
A dramatic oil painting of a determined climber ascending a treacherous snow-covered mountain peak. The man, dressed in sturdy mountaineering gear with ice axe in hand, is shown mid-struggle, muscles straining against the steep gradient. The scene is set during golden hour, with the low sun casting long shadows and illuminating the mountain face with warm orange and pink hues against the cool blues of the snow. The painting should showcase visible brushstrokes, rich textures in the clothing and snow, and 

In [20]:
import os
import logging
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv('/content/.env')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import required libraries
import dspy
from dspy import Example, Predict, ChainOfThought

# API configuration
API_KEY = os.getenv("OPENROUTER_API_KEY") or ""
MODEL = "z-ai/glm-4.5-air:free"
TEMPERATURE = 0.7

# Check if API key is available
if not API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")

# Set up DSPy to use GLM via OpenRouter
lm = dspy.LM(
    model="openai/" + MODEL,
    api_key=API_KEY,
    temperature=TEMPERATURE,
    base_url="https://openrouter.ai/api/v1"
)
dspy.settings.configure(lm=lm)
logger.info(f"Successfully configured DSPy with {MODEL}")

# Import validation functions
def validate_taste(taste: str) -> str:
    """Validate the artistic taste category."""
    valid_tastes = ['photorealistic', 'oil painting', 'anime', 'cyberpunk', 'watercolor', '3d render']
    if taste.lower() not in valid_tastes:
        raise ValueError(f"Invalid taste. Choose from: {', '.join(valid_tastes)}")
    return taste.lower()

def validate_input(user_input: str) -> str:
    """Validate the user input description."""
    if not user_input or len(user_input.strip()) < 3:
        raise ValueError("Please provide a description (at least 3 characters)")
    return user_input.strip()

# Define DSPy signatures
class InitialPromptOptimization(dspy.Signature):
    """Optimize user input into a detailed image prompt for the specified artistic style."""
    taste = dspy.InputField(desc="Artistic style (e.g., photorealistic, anime)")
    user_input = dspy.InputField(desc="Simple user description")
    optimized_prompt = dspy.OutputField(desc="Detailed optimized prompt")

class PromptEnhancement(dspy.Signature):
    """Enhance an optimized prompt with technical details and artistic refinements."""
    base_prompt = dspy.InputField(desc="Already optimized prompt")
    taste = dspy.InputField(desc="Artistic style")
    enhanced_prompt = dspy.OutputField(desc="Enhanced prompt with technical details and artistic refinements")

# Define DSPy modules
class InitialPromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.optimize = dspy.Predict(InitialPromptOptimization)

    def forward(self, taste: str, user_input: str):
        return self.optimize(taste=taste, user_input=user_input)

class PromptEnhancer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.enhance = dspy.Predict(PromptEnhancement)

    def forward(self, base_prompt: str, taste: str):
        return self.enhance(base_prompt=base_prompt, taste=taste)

class FullPromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.initial_optimizer = InitialPromptOptimizer()
        self.enhancer = PromptEnhancer()

    def forward(self, taste: str, user_input: str):
        # Step 1: Initial prompt optimization
        optimized_result = self.initial_optimizer(taste=taste, user_input=user_input)

        # Step 2: Enhance the optimized prompt
        enhanced_result = self.enhancer(
            base_prompt=optimized_result.optimized_prompt,
            taste=taste
        )

        # Return the results as attributes of a simple object
        class OptimizationResult:
            def __init__(self, optimized_prompt, enhanced_prompt):
                self.optimized_prompt = optimized_prompt
                self.enhanced_prompt = enhanced_prompt

        return OptimizationResult(
            optimized_prompt=optimized_result.optimized_prompt,
            enhanced_prompt=enhanced_result.enhanced_prompt
        )

# Prepare training data
def create_training_data() -> List[Dict[str, str]]:
    """Create training examples for DSPy."""
    return [
        {
            'taste': 'photorealistic',
            'user_input': 'a cat sitting on a windowsill',
            'optimized_prompt': 'Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture, sharp focus, natural lighting, 85mm lens, shallow depth of field',
            'enhanced_prompt': 'Ultra realistic photograph of a ginger cat sitting on a sunlit windowsill, detailed fur texture, sharp focus, natural lighting, 85mm lens, shallow depth of field, 4K resolution, f/1.8 aperture, golden hour glow, cinematic composition'
        },
        {
            'taste': 'oil painting',
            'user_input': 'a mountain landscape',
            'optimized_prompt': 'Oil painting of majestic mountain landscape at sunset, impressionist style, visible brushstrokes, warm golden hour light, textured canvas, rich color palette',
            'enhanced_prompt': 'Oil painting of majestic mountain landscape at sunset, impressionist style, visible brushstrokes, warm golden hour light, textured canvas, rich color palette, canvas texture visible, Rembrandt lighting, thick impasto technique, 24"x36" aspect ratio'
        },
        {
            'taste': 'anime',
            'user_input': 'a futuristic city',
            'optimized_prompt': 'Anime style illustration of a futuristic cyberpunk cityscape at night, neon lights, flying vehicles, detailed architecture, vibrant colors, high contrast',
            'enhanced_prompt': 'Anime style illustration of a futuristic cyberpunk cityscape at night, neon lights, flying vehicles, detailed architecture, vibrant colors, high contrast, Studio Ghibli influences, detailed backgrounds, cel-shaded rendering, 1080p resolution'
        },
        {
            'taste': 'cyberpunk',
            'user_input': 'a street market',
            'optimized_prompt': 'Cyberpunk street market at night, neon signs, diverse crowd, stalls selling futuristic tech, rain-slicked streets, Blade Runner aesthetic',
            'enhanced_prompt': 'Cyberpunk street market at night, neon signs, diverse crowd, stalls selling futuristic tech, rain-slicked streets, Blade Runner aesthetic, cinematic lighting, volumetric fog, reflections on wet pavement, ultra-detailed, 8K resolution'
        }
    ]

# Simple data container
class TrainingData:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return Example(
            taste=item['taste'],
            user_input=item['user_input'],
            optimized_prompt=item['optimized_prompt'],
            enhanced_prompt=item['enhanced_prompt']
        ).with_inputs('taste', 'user_input')

# Quality metric for DSPy evaluation
def prompt_quality_metric(example, pred, trace=None) -> float:
    """Calculate quality score for the final enhanced prompt."""
    prompt = pred.enhanced_prompt

    # Quality indicators
    has_details = 'detailed' in prompt.lower()
    has_style = any(style in prompt.lower() for style in ['style', 'technique', 'aesthetic'])
    has_technical = any(tech in prompt.lower() for tech in ['lighting', 'focus', 'texture', 'composition', 'resolution', 'aperture'])
    has_mood = any(mood in prompt.lower() for mood in ['mood', 'atmosphere', 'ambiance', 'feeling'])

    # Check if it includes relevant technical details for the style
    has_photography_terms = True
    has_painting_terms = True

    if example.taste == 'photorealistic':
        has_photography_terms = any(term in prompt.lower() for term in ['aperture', 'f/', 'shutter', 'depth', 'bokeh'])
    elif example.taste == 'oil painting':
        has_painting_terms = any(term in prompt.lower() for term in ['brushstroke', 'impasto', 'canvas', 'palette', 'texture'])

    # Calculate score
    indicators = [has_details, has_style, has_technical, has_mood, has_photography_terms, has_painting_terms]
    return sum(indicators) / len(indicators)

# Main DSPy-powered prompt generator
class DSPyPromptGenerator:
    def __init__(self):
        # Create training data
        self.trainset = TrainingData(create_training_data())

        # Initialize the full optimizer
        self.optimizer = FullPromptOptimizer()

        # Try to use BootstrapFewShot if available
        try:
            from dspy.teleprompter import BootstrapFewShot
            teleprompter = BootstrapFewShot(metric=prompt_quality_metric)
            self.compiled_optimizer = teleprompter.compile(self.optimizer, trainset=self.trainset)
            logger.info("Using compiled optimizer with BootstrapFewShot")
        except ImportError:
            try:
                from dspy.teleprompters import BootstrapFewShot
                teleprompter = BootstrapFewShot(metric=prompt_quality_metric)
                self.compiled_optimizer = teleprompter.compile(self.optimizer, trainset=self.trainset)
                logger.info("Using compiled optimizer with BootstrapFewShot")
            except ImportError:
                logger.warning("Could not import BootstrapFewShot from teleprompter module")
                logger.info("Using non-compiled optimizer instead")
                self.compiled_optimizer = self.optimizer

        self.results = []

    def generate(self, taste: str, user_input: str) -> str:
        """Generate an enhanced image prompt using DSPy-optimized modules."""
        try:
            # Validate inputs
            taste = validate_taste(taste)
            user_input = validate_input(user_input)
            logger.info(f"Validated input: {taste} - {user_input}")

            # Use the compiled DSPy optimizer or fallback
            logger.info("Running DSPy-optimized prompt generation...")
            result = self.compiled_optimizer(taste=taste, user_input=user_input)

            optimized_prompt = result.optimized_prompt
            final_prompt = result.enhanced_prompt

            # Calculate quality score
            # Create a simple object with the enhanced prompt for the metric function
            class SimplePrediction:
                def __init__(self, enhanced_prompt):
                    self.enhanced_prompt = enhanced_prompt

            quality_score = prompt_quality_metric(
                Example(taste=taste, user_input=user_input),
                SimplePrediction(final_prompt)
            )

            # Store result
            self.results.append({
                'taste': taste,
                'user_input': user_input,
                'optimized_prompt': optimized_prompt,
                'final_prompt': final_prompt,
                'quality_score': quality_score
            })

            logger.info(f"Generated prompt with quality score: {quality_score:.2f}")
            return final_prompt

        except Exception as e:
            logger.error(f"Error in prompt generation: {e}")
            fallback_prompt = f"{taste} style image of {user_input}, detailed, high quality"
            logger.info(f"Using fallback prompt: {fallback_prompt}")

            # Also store fallback result
            self.results.append({
                'taste': taste,
                'user_input': user_input,
                'optimized_prompt': fallback_prompt,
                'final_prompt': fallback_prompt,
                'quality_score': 0.0
            })

            return fallback_prompt

    def get_results(self) -> List[Dict[str, Any]]:
        """Get all generated results."""
        return self.results

def main():
    """Main function using actual DSPy."""
    print("=== DSPy-Optimized Image Prompt Generation System ===\n")
    print("Using DSPy with:")
    print("1. Defined signatures for initial optimization and enhancement")
    print("2. Multi-stage pipeline (Initial Optimizer → Enhancer)")

    try:
        from dspy.teleprompter import BootstrapFewShot
        print("3. BootstrapFewShot for automatic prompt optimization")
    except ImportError:
        try:
            from dspy.teleprompters import BootstrapFewShot
            print("3. BootstrapFewShot for automatic prompt optimization")
        except ImportError:
            print("3. Basic prompt optimization (teleprompter module not available)")

    print("4. Quality-based metrics for evaluation")
    print(f"5. GLM-4.5-Air as the backend language model\n")

    # Test examples
    examples = [
        {
            "taste": "oil painting",
            "user_input": "diwali greetings with text place holder, lantern, diya, indian festival"
        }
    ]

    # Initialize the DSPy prompt generator
    generator = DSPyPromptGenerator()

    for i, example in enumerate(examples, 1):
        print(f"Example {i}:")
        print(f"Taste: {example['taste']}")
        print(f"User Input: {example['user_input']}")

        try:
            # Generate using DSPy
            result = generator.generate(
                taste=example['taste'],
                user_input=example['user_input']
            )

            print(f"\nDSPy-Optimized Prompt:")
            print(f"{generator.results[-1]['optimized_prompt']}")

            print(f"\nDSPy-Enhanced Prompt (Final Output):")
            print(f"{result}")

            print(f"\nQuality Score: {generator.results[-1]['quality_score']:.2f}")
            print("✅ SUCCESS: DSPy-optimized prompt generation completed!\n")

        except Exception as e:
            print(f"❌ FAILED: {e}\n")

        print("="*70)

if __name__ == "__main__":
    main()

=== DSPy-Optimized Image Prompt Generation System ===

Using DSPy with:
1. Defined signatures for initial optimization and enhancement
2. Multi-stage pipeline (Initial Optimizer → Enhancer)
3. Basic prompt optimization (teleprompter module not available)
4. Quality-based metrics for evaluation
5. GLM-4.5-Air as the backend language model

Example 1:
Taste: oil painting
User Input: diwali greetings with text place holder, lantern, diya, indian festival

DSPy-Optimized Prompt:
An elegant oil painting of Diwali festival celebration, featuring traditional Indian oil lamps (diyas) glowing softly, colorful paper lanterns hanging in the background, with a decorative text placeholder for 'Happy Diwali' in the foreground. The painting should showcase rich, warm colors in deep reds, golds, and oranges characteristic of Diwali. Visible brushstrokes and textured canvas should be evident, capturing the festive atmosphere with a slightly dreamy quality. The composition should balance the elements ha